# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 8:30AM,242372,10,0085889862,ISDIN Corporation,Released
1,Jul 21 2022 8:30AM,242372,10,0085893476,ISDIN Corporation,Released
2,Jul 21 2022 8:30AM,242372,10,0085893494,ISDIN Corporation,Released
3,Jul 21 2022 8:30AM,242372,10,0085893493,ISDIN Corporation,Released
4,Jul 21 2022 8:30AM,242371,10,0085893508,ISDIN Corporation,Released
5,Jul 21 2022 8:30AM,242372,10,0085893512,ISDIN Corporation,Released
6,Jul 21 2022 8:30AM,242372,10,0085893513,ISDIN Corporation,Released
7,Jul 21 2022 8:30AM,242372,10,0085893514,ISDIN Corporation,Released
8,Jul 21 2022 8:30AM,242372,10,0085893510,ISDIN Corporation,Released
9,Jul 21 2022 8:30AM,242372,10,0085893511,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,242365,Released,23
26,242367,Released,25
27,242371,Released,1
28,242372,Released,34
29,242373,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
242365,NaN,NaN,23.0
242367,NaN,NaN,25.0
242371,NaN,NaN,1.0
242372,NaN,NaN,34.0
242373,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242281,4.0,23.0,5.0
242282,0.0,0.0,1.0
242290,0.0,0.0,1.0
242295,0.0,0.0,1.0
242297,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
242281,4,23,5
242282,0,0,1
242290,0,0,1
242295,0,0,1
242297,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242281,4,23,5
1,242282,0,0,1
2,242290,0,0,1
3,242295,0,0,1
4,242297,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,242281,4,23,5
1,242282,,,1
2,242290,,,1
3,242295,,,1
4,242297,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 8:30AM,242372,10,ISDIN Corporation
4,Jul 21 2022 8:30AM,242371,10,ISDIN Corporation
35,Jul 20 2022 4:15PM,242373,19,"GUSA Granules USA, Inc."
36,Jul 20 2022 3:24PM,242367,10,ISDIN Corporation
61,Jul 20 2022 3:24PM,242365,10,ISDIN Corporation
84,Jul 20 2022 3:24PM,242332,10,ISDIN Corporation
122,Jul 20 2022 2:32PM,242359,10,Bio-PRF
130,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd"
175,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc."
176,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jul 21 2022 8:30AM,242372,10,ISDIN Corporation,,,34
1,Jul 21 2022 8:30AM,242371,10,ISDIN Corporation,,,1
2,Jul 20 2022 4:15PM,242373,19,"GUSA Granules USA, Inc.",,,1
3,Jul 20 2022 3:24PM,242367,10,ISDIN Corporation,,,25
4,Jul 20 2022 3:24PM,242365,10,ISDIN Corporation,,,23
5,Jul 20 2022 3:24PM,242332,10,ISDIN Corporation,,,38
6,Jul 20 2022 2:32PM,242359,10,Bio-PRF,,,8
7,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd",,29,16
8,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",,,1
9,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 8:30AM,242372,10,ISDIN Corporation,34,,
1,Jul 21 2022 8:30AM,242371,10,ISDIN Corporation,1,,
2,Jul 20 2022 4:15PM,242373,19,"GUSA Granules USA, Inc.",1,,
3,Jul 20 2022 3:24PM,242367,10,ISDIN Corporation,25,,
4,Jul 20 2022 3:24PM,242365,10,ISDIN Corporation,23,,
5,Jul 20 2022 3:24PM,242332,10,ISDIN Corporation,38,,
6,Jul 20 2022 2:32PM,242359,10,Bio-PRF,8,,
7,Jul 20 2022 1:32PM,242345,19,"AdvaGen Pharma, Ltd",16,29,
8,Jul 20 2022 12:57PM,242340,21,"NBTY Global, Inc.",1,,
9,Jul 20 2022 12:51PM,242338,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 8:30AM,242372,10,ISDIN Corporation,34,,
1,Jul 21 2022 8:30AM,242371,10,ISDIN Corporation,1,,
2,Jul 20 2022 4:15PM,242373,19,"GUSA Granules USA, Inc.",1,,
3,Jul 20 2022 3:24PM,242367,10,ISDIN Corporation,25,,
4,Jul 20 2022 3:24PM,242365,10,ISDIN Corporation,23,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 8:30AM,242372,10,ISDIN Corporation,34.0,NaN,NaN
1,Jul 21 2022 8:30AM,242371,10,ISDIN Corporation,1.0,NaN,NaN
2,Jul 20 2022 4:15PM,242373,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN
3,Jul 20 2022 3:24PM,242367,10,ISDIN Corporation,25.0,NaN,NaN
4,Jul 20 2022 3:24PM,242365,10,ISDIN Corporation,23.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jul 21 2022 8:30AM,242372,10,ISDIN Corporation,34.0,0.0,0.0
1,Jul 21 2022 8:30AM,242371,10,ISDIN Corporation,1.0,0.0,0.0
2,Jul 20 2022 4:15PM,242373,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0
3,Jul 20 2022 3:24PM,242367,10,ISDIN Corporation,25.0,0.0,0.0
4,Jul 20 2022 3:24PM,242365,10,ISDIN Corporation,23.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2665684,138.0,24.0,4.0
12,242336,1.0,0.0,0.0
16,969164,4.0,0.0,0.0
19,1454007,21.0,31.0,0.0
21,726984,3.0,0.0,0.0
22,242329,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2665684,138.0,24.0,4.0
1,12,242336,1.0,0.0,0.0
2,16,969164,4.0,0.0,0.0
3,19,1454007,21.0,31.0,0.0
4,21,726984,3.0,0.0,0.0
5,22,242329,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,138.0,24.0,4.0
1,12,1.0,0.0,0.0
2,16,4.0,0.0,0.0
3,19,21.0,31.0,0.0
4,21,3.0,0.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,138.0
1,12,Released,1.0
2,16,Released,4.0
3,19,Released,21.0
4,21,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,21,22
Status,,,,,,
Completed,4.0,0.0,0.0,0.0,0.0,0.0
Executing,24.0,0.0,0.0,31.0,0.0,0.0
Released,138.0,1.0,4.0,21.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,21,22
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0
1,Executing,24.0,0.0,0.0,31.0,0.0,0.0
2,Released,138.0,1.0,4.0,21.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,21,22
0,Completed,4.0,0.0,0.0,0.0,0.0,0.0
1,Executing,24.0,0.0,0.0,31.0,0.0,0.0
2,Released,138.0,1.0,4.0,21.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()